## Programming Gaussian

A gaussian function can be expressed as

$$f(x) = \frac{1}{\sqrt{2\pi\sigma^2}} 
       \exp\!\left( -\frac{(x - \mu)^2}{2\sigma^2} \right)$$

where

$\mu$ = mean (center of the distribution)

$\sigma$ = standard deviation (spread/width)

$exp$ = exponential function

### Create python function to generate gaussian values of input

In [4]:
from math import *

def f(mu, sigma2,  x):
    return (1/sqrt(2*pi* sigma2)) * exp((x - mu)**2 / (-2 * sigma2))

In [5]:
f(10,4., 10.)

0.19947114020071635

# Kalman Filter

## Steps

Kalman Filter involves a measurement update step, and a motion update(prediction) step

Measurement Update -> using Bayesian filter (Product)

Motion Update -> using Total Probability (Convolution)

## Measurement Update

Measurement Update is the product of two gaussian probability distributions.

These are prior and measurement gaussian distributions, each having its own mean $\mu$ and variance $\sigma$

### Product of Gaussian distributions

The resultant mean is a precision-weighted average of contributing gaussian distributions.

Precision $\mathbf{p}$ is the inverse of variance $\mathbf{\sigma^2}$, i.e., $$p = \frac{1}{\sigma^2}$$

Resultan mean **in terms of precision:**

$$\mu = \frac{p_1\mu_1 + p_2\mu_2}{p_1 + p_2}$$
Resultant mean **in terms of variance:**
$$\mu = \frac{\sigma_2^2\mu_1 + \sigma_1^2\mu_2}{\sigma_1^2 + \sigma_2^2}$$

Resultant Variance:
$$\sigma^2 = \frac{1}{\frac{1}{\sigma_1^2} + \frac{1}{\sigma_2^2} } = \frac{\sigma_1^2\sigma_2^2}{\sigma_1^2 + \sigma_2^2}$$

Resultant Precision:
$$p = p_1 + p_2$$

### Create function to combine (product) of two Gaussians

In [6]:
def update(mean1, var1, mean2, var2):
    mean = (var2*mean1 +var1*mean2)/(var1+var2)
    var = 1/((1/var1) + (1/var2))
    return [mean, var]

In [7]:
update(10.,4.,12.,4)

[11.0, 2.0]

## Motion Update

It is the change in the gaussian probability distribution (of where the robot is), on moving the robot.
This is because the robot movement itself is gaussian with motion having a mean $\nu$ and an uncertainty $\gamma^2$

The resulting prob distribution has a mean
$$\mu_{n+1} = \mu_n + \nu$$

The variance of the resulting distribution is what
$$\sigma_{n+1}^2 = \sigma_n^2 + \gamma^2$$

In [8]:
def predict(mean1, var1, mean2, var2):
    return [mean1+mean2, var1+var2]

In [9]:
predict(10.,4., 12., 4.)

[22.0, 8.0]

## Implementation

In [10]:
# Example
measurements = [5., 6., 7., 9., 10.]
motion = [1., 1., 2., 1., 1.]

# variances
measurement_sig = 4.
motion_sig = 2.

# initial belief
mu = 0. # wrong estimate
sig = 10000. # large uncertainty

sig=0.00000001 

In [11]:
def kalman_basic(initial_belief, z_sig, u_sig, z_list, u_list):
    mu, sig = initial_belief
    for i in range(len(z_list)):
        z = z_list[i]
        u = u_list[i]
        # measurement update
        mu,sig = update(mu, sig, z, z_sig)
        print(f"update:\t[{mu:.12f},\t{sig:.12f}]")
        # motion update
        mu, sig = predict(mu, sig, u, u_sig)
        print(f"predict:[{mu:.12f},\t{sig:.12f}]")
    

In [12]:
kalman_basic([mu, sig], measurement_sig, motion_sig, measurements, motion)

update:	[0.000000012500,	0.000000010000]
predict:[1.000000012500,	2.000000010000]
update:	[2.666666680556,	1.333333337778]
predict:[3.666666680556,	3.333333337778]
update:	[5.181818190496,	1.818181819504]
predict:[7.181818190496,	3.818181819504]
update:	[8.069767446458,	1.953488372439]
predict:[9.069767446458,	3.953488372439]
update:	[9.532163745022,	1.988304093655]
predict:[10.532163745022,	3.988304093655]
